In [1]:
# muzero in a notebook on cartpole

In [34]:
%pylab inline
import tensorflow as tf
#import tensorflow.keras.backend as K
import numpy as np
import gym
from tqdm import trange
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import *
np.set_printoptions(suppress=True)

# toh -> to_one_hot
def toh(x,n):
  ret = np.zeros([n])
  ret[x] = 1.0
  return ret

def bstack(bb):
  ret = [[x] for x in bb[0]]
  for i in range(1, len(bb)):
    for j in range(len(bb[i])):
      ret[j].append(bb[i][j])
  return [np.array(x) for x in ret]

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['subtract', 'dot', 'f', 'average', 'add', 'gamma', 'minimum', 'concatenate', 'maximum', 'multiply']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
env = gym.make("CartPole-v0")
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [4]:
S_DIM = 4

# h: representation function
# s_0 = h(o_1...o_t)
x = o_0 = Input(env.observation_space.shape)
x = Dense(S_DIM)(x)
s_0 = Activation('tanh')(x)
unh = Model(o_0, s_0, name="h")
def ht(o_0):
  return h.predict(o_0[None])[0]

# g: dynamics function (recurrent in state?) old_state+action -> state+reward
# r_k, s_k = g(s_k-1, a_k)
s_km1 = Input(S_DIM)
a_k = Input(env.action_space.n)
x = Concatenate()([s_km1, a_k])
x = Dense(64)(x)
x = Activation('elu')(x)
x = Dense(64)(x)
x = Activation('elu')(x)
s_k = Dense(S_DIM, name='s_k')(x)
r_k = Dense(1, name='r_k')(x)
g = Model([s_km1, a_k], [r_k, s_k], name="g")
g.compile('adam', 'mse')
def gt(s_km1, a_k):
  r_k, s_k = g.predict([s_km1[None], a_k[None]])
  return r_k[0], s_k[0]

# f: prediction function -- state -> policy+value
# p_k, v_k = f(s_k)
x = s_k = Input(S_DIM)
x = Dense(32)(x)
x = Activation('tanh')(x)
p_k = Dense(env.action_space.n)(x)
p_k = Activation('softmax', name='p_k')(p_k)
v_k = Dense(1, name='v_k')(x)
f = Model(s_k, [p_k, v_k], name="f")
f.compile('adam', 'mse')
def ft(s_k):
  p_k, v_k = f.predict(s_k[None])
  return p_k[0], v_k[0]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
# it's using the dynamics function for rollout search
K = 5
gamma = 0.95

# represent
o_0 = Input(env.observation_space.shape, name="o_0")
# don't use the h function for now
#s_km1 = h(o_0)
s_km1 = o_0

# rollout with dynamics
# p_k, v_k, r_k = mu(o_0, a_1_k)
a_all, mu_all = [], []
for k in range(K):
  a_k = Input(env.action_space.n, name="a_%d" % k)
  r_k, s_k  = g([s_km1, a_k])

  # predict
  p_k, v_k = f([s_k])

  # store
  a_all.append(a_k)
  mu_all.append([p_k, v_k, r_k])
  s_km1 = s_k

# put in the first observation and actions
#   need policy from search
#   need values from sum of rewards + last state value (real state?)
#   need rewards
#a_all = Concatenate()(a_all)
mu = Model([o_0, a_all], mu_all)
mu.compile('adam', 'mse')
mu.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
o_0 (InputLayer)                [(None, 4)]          0                                            
__________________________________________________________________________________________________
a_0 (InputLayer)                [(None, 2)]          0                                            
__________________________________________________________________________________________________
g (Model)                       [(None, 1), (None, 4 4933        o_0[0][0]                        
                                                                 a_0[0][0]                        
                                                                 g[1][1]                          
                                                                 a_1[0][0]                    

In [53]:
import itertools

aopts = list(itertools.product([0,1], repeat=K))
aoptss = np.array([[toh(x, 2) for x in aa] for aa in aopts])
aoptss = aoptss.swapaxes(0,1)
aoptss = [aoptss[x] for x in range(5)]
# every possible action for the next 5 time steps

def search(o_0):
  # uses the whole mu function
  # 2^5 rollout isn't too bad, explore them all
  # maximize value
  
  # concatenate the current state with every possible action
  o_0s = np.repeat(np.array(o_0)[None], len(aopts), axis=0)
  # ret is p_0, v_0, r_0, p_1, v_1, r_1, ...
  ret = mu.predict([o_0s]+aoptss)
  v_s = ret[-2]
  
  # group the value with the action rollout that caused it
  v = [(v_s[i][0], aopts[i]) for i in range(len(v_s))]
  return sorted(v, reverse=True)

env.reset()
search(env.state)

[(13.34436, (0, 1, 0, 1, 1)),
 (13.338605, (0, 1, 1, 0, 1)),
 (13.334851, (1, 0, 0, 1, 1)),
 (13.27295, (0, 0, 1, 1, 1)),
 (13.243852, (0, 1, 1, 1, 0)),
 (13.225421, (1, 0, 1, 0, 1)),
 (13.018701, (1, 0, 1, 1, 0)),
 (13.005616, (1, 1, 0, 0, 1)),
 (12.78478, (1, 1, 0, 0, 0)),
 (12.67726, (1, 1, 0, 1, 0)),
 (12.494369, (1, 0, 1, 0, 0)),
 (12.248636, (1, 1, 1, 0, 0)),
 (12.128003, (1, 0, 0, 1, 0)),
 (12.112628, (0, 1, 1, 0, 0)),
 (11.752294, (1, 0, 0, 0, 1)),
 (11.696443, (0, 1, 0, 1, 0)),
 (11.30598, (0, 0, 1, 1, 0)),
 (11.283534, (0, 1, 0, 0, 1)),
 (10.997111, (0, 1, 1, 1, 1)),
 (10.872046, (0, 0, 1, 0, 1)),
 (10.494156, (0, 0, 0, 1, 1)),
 (10.44582, (1, 0, 1, 1, 1)),
 (9.8302, (1, 1, 0, 1, 1)),
 (9.186256, (1, 1, 1, 0, 1)),
 (8.71386, (1, 0, 0, 0, 0)),
 (8.567235, (1, 1, 1, 1, 0)),
 (8.201274, (0, 1, 0, 0, 0)),
 (7.7830534, (0, 0, 1, 0, 0)),
 (7.421197, (0, 0, 0, 1, 0)),
 (7.10236, (0, 0, 0, 0, 1)),
 (6.2879, (1, 1, 1, 1, 1)),
 (4.9739814, (0, 0, 0, 0, 0))]

In [7]:
env.reset()
for _ in range(10):
  x,y = [],[]
  for _ in range(16):
    o_0 = np.copy(env.state)
    acts = search(o_0)
    v_k, a_0k = acts[0]
    #v_k, a_0k = acts[random.randint(0, len(acts))]
    a_0koh = [toh(x, 2) for x in a_0k]
    x.append([o_0]+a_0koh)

    # actually act with best value policy
    # TODO: Shouldn't I only take one step?
    rs = []
    for i in range(K):
      _, r, done, _ = env.step(a_0k[i])
      if done:
        #r = -50
        env.reset()
      rs.append(r)

    # compute values
    v_0k = [v_k]
    for r in rs[::-1]:
      v_0k = [v_0k[0]*gamma + r] + v_0k

    yl = []
    for i in range(K):
      yl += [a_0koh[i], v_0k[i], rs[i]]
    y.append(yl)

  ll = mu.fit(bstack(x), bstack(y), verbose=0)
  loss = ll.history['loss']
  print(loss)

[59.013423919677734]
[56.43766784667969]
[54.858863830566406]
[53.248355865478516]
[51.649967193603516]
[50.58441925048828]
[48.937862396240234]
[48.761783599853516]
[48.043514251708984]
[48.119144439697266]


In [40]:
# can act?
# sort of
env.reset()
for sn in range(100):
  ret = search(env.state)
  v,aa = ret[0]
  print(aa[0], v)
  env.render()
  _,r,done,_ = env.step(aa[0])
  if done:
    print("DONE", sn)
    break

1 13.476954
1 13.377585
0 13.455794
0 13.377248
0 13.455531
1 13.395326
0 13.417607
1 13.409043
0 13.3715315
1 13.401563
0 13.284837
1 13.415082
1 13.222312
0 13.390389
1 13.239273
0 13.181138
0 13.251421
0 13.367643
1 13.279827
0 13.302781
1 13.2988825
0 13.20173
1 13.304494
0 13.091646
1 13.306598
1 13.104393
0 13.221051
1 13.125908
0 12.965182
0 13.13664
0 13.274758
1 13.149135
0 13.265711
1 13.177405
0 13.254174
1 13.200692
0 13.256652
1 13.201125
0 13.259032
1 13.160111
0 13.270404
1 13.063179
0 13.282321
0 13.087555
1 13.163037
1 13.107883
1 13.324465
0 13.113534
1 13.336276
0 13.172433
1 13.305646
1 13.233598
1 13.372317
0 13.288291
0 13.376298
0 13.317983
1 13.376091
0 13.304309
1 13.387071
0 13.297696
1 13.388679
0 13.276006
1 13.390489
0 13.225219
1 13.383586
1 13.145783
0 13.381992
1 13.215787
0 13.314189
0 13.235341
0 13.241827
0 13.264119
0 13.1426935
1 13.2724085
1 13.011491
0 13.243276
1 13.108466
0 13.211682
1 13.145431
0 13.160335
1 13.152785
0 13.04826
1 13.143466
1 1

In [26]:
# Test: Can the f function learn a policy+value model
# This isn't a policy model at all! The policy is being randomly sampled
# The values should be correct though

x_s = []
y_p = []
y_v = []

for _ in trange(1000):
  env.reset()
  rs = []
  for _ in range(100):
    a_k = env.action_space.sample()
    _, r, done, _ = env.step(a_k)
    x_s.append(env.state)
    y_p.append(toh(a_k, 2))
    rs.append(r)
  v_0k = [0]
  for r in rs[::-1]:
    v_0k = [v_0k[0]*gamma + r] + v_0k
  y_v += v_0k[:-1]
    
x_s = np.array(x_s)
y_p, y_v = np.array(y_p), np.array(y_v)
    
f.fit(x_s, [y_p, y_v], batch_size=32, epochs=5, shuffle=1)

100%|██████████| 1000/1000 [00:02<00:00, 416.82it/s]


Train on 100000 samples
Epoch 1/5
100000/100000 [==============================] - 3s 28us/sample - loss: 6.6475 - p_k_loss: 0.2451 - v_k_loss: 6.4025
Epoch 2/5
100000/100000 [==============================] - 3s 26us/sample - loss: 2.7865 - p_k_loss: 0.2418 - v_k_loss: 2.5447
Epoch 3/5
100000/100000 [==============================] - 3s 26us/sample - loss: 2.4166 - p_k_loss: 0.2401 - v_k_loss: 2.1765
Epoch 4/5
100000/100000 [==============================] - 3s 26us/sample - loss: 2.3383 - p_k_loss: 0.2389 - v_k_loss: 2.09940s - loss: 2.3353 - p_k_loss: 0.2389 - v_k_loss
Epoch 5/5
100000/100000 [==============================] - 3s 26us/sample - loss: 2.3123 - p_k_loss: 0.2369 - v_k_loss: 2.07540s - loss: 2.3129 - p_k_loss: 0.2369 - v_k_loss: 2.076


In [36]:
# Test: Can the g function learn a dynamics model
# Answer: yes!
# Sadly, with h, this is hard to test in the whole system
# With h, this training also doesn't work

x_s, x_a = [], []
y_r, y_s = [], []

for _ in trange(1000):
  env.reset()
  for _ in range(100):
    #env.render()
    a_k = env.action_space.sample()
    x_s.append(np.copy(env.state))
    x_a.append(toh(a_k, env.action_space.n))
    _, r, done, _ = env.step(a_k)
    y_r.append([r])
    y_s.append(np.copy(env.state))
    #if done:
    #    break
  env.close()
x_s, x_a = np.array(x_s), np.array(x_a)
y_r, y_s = np.array(y_r), np.array(y_s)

g.fit([x_s, x_a], [y_r, y_s], batch_size=32, epochs=5, shuffle=1)

100%|██████████| 1000/1000 [00:03<00:00, 302.90it/s]


Train on 100000 samples
Epoch 1/5
100000/100000 [==============================] - 3s 32us/sample - loss: 0.0161 - r_k_loss: 0.0125 - s_k_loss: 0.0036
Epoch 2/5
100000/100000 [==============================] - 3s 32us/sample - loss: 0.0153 - r_k_loss: 0.0120 - s_k_loss: 0.0033
Epoch 3/5
100000/100000 [==============================] - 3s 33us/sample - loss: 0.0146 - r_k_loss: 0.0116 - s_k_loss: 0.0030
Epoch 4/5
100000/100000 [==============================] - 3s 32us/sample - loss: 0.0139 - r_k_loss: 0.0110 - s_k_loss: 0.0028
Epoch 5/5
100000/100000 [==============================] - 3s 32us/sample - loss: 0.0134 - r_k_loss: 0.0107 - s_k_loss: 0.0027


In [27]:
# test the dynamics model
env.reset()
for _ in range(100):
  env.render()
  a_k = env.action_space.sample()
  r_k, s_k = gt(env.state, toh(a_k, env.action_space.n))
  p_k, v_k = ft(s_k)
  print(a_k, r_k, v_k, np.argmax(p_k))
  env.unwrapped.state = s_k
  #env.step(a_k)
#env.close()

0 [1.0913224] [13.260438] 0
1 [1.1019825] [13.559748] 1
1 [1.1110841] [13.388826] 1
1 [1.1218592] [12.557975] 1
1 [1.1337718] [11.115157] 1
1 [1.1442887] [9.338042] 1
0 [1.1341516] [11.152249] 1
0 [1.1388054] [13.05694] 1
1 [1.1333196] [12.2041445] 1
1 [1.1461475] [10.959494] 1
0 [1.1582664] [13.077177] 1
0 [1.1283102] [13.024298] 1
1 [1.0266144] [13.205422] 1
1 [1.0074905] [13.237667] 1
0 [1.0173914] [11.31866] 1
1 [0.80051416] [11.845709] 1
1 [0.7863646] [12.393792] 1
1 [0.8078353] [12.759163] 1
0 [0.88486814] [10.547026] 1
0 [0.71679336] [6.8668246] 1
0 [0.5788619] [3.6401303] 1
1 [0.44443744] [3.5394957] 1
0 [0.43859982] [1.4510083] 1
1 [0.33644825] [1.1293046] 1
1 [0.3175444] [1.0383422] 1
1 [0.32039374] [1.2388549] 1
0 [0.34238672] [0.26474696] 1
1 [0.32169968] [0.30323267] 1
0 [0.33577132] [-0.15327317] 1
1 [0.35152853] [-0.14448422] 1
0 [0.37289697] [-0.2847427] 1
0 [0.40749806] [-0.3113476] 1
1 [0.52140427] [-0.3340394] 1
1 [0.67884254] [0.00528491] 1
1 [0.89108616] [1.7626506